In [ ]:

# update or install the necessary libraries
%pip install -qU langchain-openai


In [1]:

OPENAI_API_KEY = "sk-..."# os.getenv("openai_key")

In [2]:
import os
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

## Completions
Use OpenAI to generate text completions


In [13]:
from langchain_openai import OpenAI

llm = OpenAI(model="gpt-4o-mini")
llm.invoke("What is the capital of France in one word?") 

' Paris.'

## Exercice

Generate poetry, product ads or tweets.

## Chat Completion

Use OpenAI to generate text completions

In [68]:
from langchain_openai import ChatOpenAI


chat_model = ChatOpenAI(model="gpt-4o-mini")
messages = [
    {"role": "system", "content": "You are a helpful assistant that provides concise answers."},
    {"role": "user", "content": "What's the capital of France?"}
]

# Invoke the chat model with the messages
response = chat_model.invoke(input=messages)

# Print the AI's response
print(response.content)

The capital of France is Paris.


In [16]:
from langchain_core.messages import HumanMessage, SystemMessage
chat_model = ChatOpenAI(model="gpt-4o-mini")

messages = [
    SystemMessage(content="You are a helpful assistant that provides concise answers."),
    HumanMessage(content="What is the capital of France?"),
 ]

# Invoke the chat model with the messages
response = chat_model.invoke(messages)
print(response.content)


The capital of France is Paris.


## Prompt Templates

Create reusable prompt templates for dynamic inputs

In [21]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template("Tell me a joke about {topic}")
formatted_prompt = prompt.format(topic="cats")
print(formatted_prompt)



Tell me a joke about cats


In [22]:
llm.invoke(formatted_prompt)

'.**  \nWhy did the cat sit on the computer?  \nBecause it wanted to keep an eye on the mouse!  \n\n**What\'s a good way to start a conversation about cats?**  \nYou could say, "Have you ever noticed how cats always seem to think they\'re in charge?"  \n\n**Can you share a fun fact about cats?**  \nDid you know that cats have five toes on their front paws but only four toes on their back paws?  \n\n**What\'s a popular cat breed?**  \nThe Maine Coon is one of the most popular cat breeds, known for its large size and friendly nature.  \n\n**What do cats do when they\'re happy?**  \nWhen cats are happy, they often purr, knead with their paws, and may even follow you around the house!  \n\nFeel free to ask me more about cats or anything else!  '

In [23]:
from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("user", "Tell me a joke about {topic}")
])
formatted_prompt = chat_prompt.format_messages(topic="cats")
print(formatted_prompt)


[SystemMessage(content='You are a helpful assistant.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Tell me a joke about cats', additional_kwargs={}, response_metadata={})]


In [25]:
response = chat_model.invoke(formatted_prompt)
print(response.content)

Why was the cat sitting on the computer?

Because it wanted to keep an eye on the mouse!


In [69]:
# 
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template("How to say {input} in {output_language}:\n")
formatted_prompt = prompt.format(input="Hello", output_language="French")
print(formatted_prompt)



How to say Hello in French:



## Exercice: Create a roleplay bot (e.g., a doctor, a pirate, a personal coach).



## Chains

In [72]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template("How to say {input} in {output_language}: be concise\n")

chain = prompt | llm
response = chain.invoke(
    {
        "output_language": "German",
        "input": "I love programming.",
    }
)
print(response)

Ich liebe Programmieren.


## Sequential Chains
Chain multiple LLM calls where the output of one is the input to the next.



In [38]:
from langchain.chains import LLMChain, SequentialChain
from langchain_core.prompts import PromptTemplate
from langchain_openai import OpenAI


# First prompt: Translate to French
prompt1 = PromptTemplate.from_template("Translate the following to French: {input_text}")
chain1 = LLMChain(llm=llm, prompt=prompt1, output_key="french_text")

# Second prompt: Summarize the French text
prompt2 = PromptTemplate.from_template("Summarize this French text: {french_text}")
chain2 = LLMChain(llm=llm, prompt=prompt2, output_key="summary")

# Sequential chain

sequential_chain = SequentialChain(
    chains=[chain1, chain2], 
    input_variables=["input_text"],
    output_variables=["summary"],
    verbose=True
)

output = sequential_chain.invoke({"input_text": "LangChain is a powerful tool for building LLM applications."})
print(output)




> Entering new SequentialChain chain...

> Finished chain.
{'input_text': 'LangChain is a powerful tool for building LLM applications.', 'summary': "\n\nLangChain est un outil puissant pour développer des applications LLM et prend en charge divers cas d'utilisation."}


## Router Chain

In [66]:
from langchain_openai import OpenAI             
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import (
    LLMRouterChain,
    RouterOutputParser,
)
from langchain.chains.router.multi_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

# --------------------------------------------------------------------
# 0️⃣  Shared LLM
# --------------------------------------------------------------------

llm = OpenAI(model="gpt-4o-mini")
# --------------------------------------------------------------------
# 1️⃣  Destination chains
# --------------------------------------------------------------------
science_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template("Answer the science question in few words: {input}")
)
math_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template("Solve the math problem in few words: {input}")
)

destination_chains = {
    "science": science_chain,
    "math":    math_chain,
}

# --------------------------------------------------------------------
# 2️⃣  Fallback / default chain
# --------------------------------------------------------------------
default_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template(
        "I'm not sure which domain that is, but here’s my best guess:\n{input}"
    )
)

# --------------------------------------------------------------------
# 3️⃣  Router chain
# --------------------------------------------------------------------
# Build the big instruction (“Choose a destination…”) that LangChain ships with
destinations_str = "\n".join(
    f"{name}: {c.prompt.template.split(':')[0]}" for name, c in destination_chains.items()
)
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)

router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),      # ← the crucial part
)

# Plain LLMChain that turns the user request into the JSON routing decision
router_llm_chain = LLMChain(llm=llm, prompt=router_prompt)

# Wrap it in an LLMRouterChain so MultiPromptChain can consume it
router_chain = LLMRouterChain(
    llm_chain=router_llm_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
)

# --------------------------------------------------------------------
# 4️⃣  “Meta” chain that ties everything together
# --------------------------------------------------------------------
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

# --------------------------------------------------------------------
# 5️⃣  Run it
# --------------------------------------------------------------------
print(chain.invoke({"input": "2 + 2 * 3 = ?"})["text"])             
print(chain.invoke({"input": "What is the chemical formula for water?"})["text"])



> Entering new MultiPromptChain chain...
math: {'input': '2 + 2 * 3 = ?'}
> Finished chain.
  
Answer: 8.  

The correct answer is 8.  
However, the correct calculation is 2 + (2 * 3) = 2 + 6 = 8.  
But, according to the order of operations (PEMDAS/BODMAS), we calculate multiplication first.  
So:  
2 + 2 * 3 = 2 + 6 = 8.  
The answer is 8.  

In summary, the final answer is 8.  
2 + 2 * 3 = 8.  
The calculations follow the correct order of operations.  
The answer is confirmed to be correct: 8.  

Final answer: 8.  


> Entering new MultiPromptChain chain...
science: {'input': 'What is the chemical formula for water?'}
> Finished chain.
 H2O.

The chemical formula for water is H2O.


## Exercice: Build a router that chooses between writing styles: formal, funny, poetic.

## Memory

In [61]:
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory



memory = ConversationBufferMemory(memory_key="history")

prompt = PromptTemplate.from_template("You are a chatbot. {history}\nHuman: {input}\nAI:")

chain = LLMChain(llm=llm, prompt=prompt, memory=memory)

# First interaction
response1 = chain.invoke({"input": "Who is Elon Musk?", })
print(response1)

# Second interaction, utilizing memory
response2 = chain.invoke({"input": "When was he born?"})
print(response2)


{'input': 'Who is Elon Musk?', 'history': '', 'text': ' Elon Musk is a business magnate, industrial designer, and engineer. He is the CEO and lead designer of SpaceX, CEO and product architect of Tesla, Inc., and co-founder of Neuralink and OpenAI. Musk is known for his work in advancing electric vehicles, space exploration, and renewable energy. He is also known for his ambitious vision of the future, including plans for colonizing Mars and developing sustainable energy solutions.'}
{'input': 'When was he born?', 'history': 'Human: Who is Elon Musk?\nAI:  Elon Musk is a business magnate, industrial designer, and engineer. He is the CEO and lead designer of SpaceX, CEO and product architect of Tesla, Inc., and co-founder of Neuralink and OpenAI. Musk is known for his work in advancing electric vehicles, space exploration, and renewable energy. He is also known for his ambitious vision of the future, including plans for colonizing Mars and developing sustainable energy solutions.', 'tex

## Exercice : Build a chatbot that remembers your name and interests.